In [1]:
import torch
import torch.nn as nn
import torchvision
from netCDF4 import Dataset
import numpy as np
import horovod.torch as hvd
from data import get_data,get_X_names
from unet import *
%load_ext autoreload
%autoreload 2
# from pytorch_model_summary import summary

In [2]:
x_names = get_X_names()
x_names = x_names[3:8] + x_names[25:30]

In [34]:
print("Processing data")
X,Y,ymu,ystd = get_data( x_names[:10],['ccn_001','ccn_003'],"data/training.nc")
print("Writing data")
torch.save(X,'data/trainX')
torch.save(Y,'data/trainY')
torch.save(ymu,'data/train_ymu')
torch.save(ystd,'data/train_ystd')

Processing data
Has 134 samples
Writing data


In [7]:
print("Processing data")
X_test,Y_test,ymu_test,ystd_test = get_data(x_names[:10],['ccn_001','ccn_003'],"data/evaluation.nc")
print("Writing data")
torch.save(X_test,'data/testX')
torch.save(Y_test,'data/testY')
torch.save(ymu_test,'data/test_ymu')
torch.save(ystd_test,'data/test_ystd')

Processing data
Has 12 samples
Writing data
